In [2]:
import pandas as pd
import random

# 샘플 데이터 생성
data = [
    "Artificial intelligence is transforming industries.",
    "Machine learning algorithms improve automatically through experience.",
    "Deep learning is a subset of machine learning.",
    "Natural language processing enables computers to understand human language.",
    "Computer vision is enabling new capabilities in image and video analysis.",
    "Reinforcement learning is an area of machine learning focused on how agents should take actions."
]

# 노이즈 추가된 데이터 생성
noisy_data = data.copy()
noisy_data.extend([
    "Random text without meaning.",
    "1234567890",
    "Garbage data that doesn't make sense.",
    "Another irrelevant sentence.",
    "Some more noise data here.",
    "Just some random words strung together."
])

# 데이터프레임으로 변환
df = pd.DataFrame(noisy_data, columns=['text'])

# 데이터 저장
df.to_csv('/home/eternal/noisy_data.csv', index=False)

import ace_tools as tools; tools.display_dataframe_to_user(name="Noisy Data", dataframe=df)


ModuleNotFoundError: No module named 'ace_tools'

In [35]:
from huggingface_hub import notebook_login
notebook_login()

In [52]:
!pip3 install -q -U transformers
!pip3 install -q -U datasets
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-cpu 2.15.1 requires flatbuffers>=23.5.26, but you have flatbuffers 1.12 which is incompatible.
tensorflow-cpu 2.15.1 requires keras<2.16,>=2.15.0, but you have keras 2.9.0 which is incompatible.
tensorflow-cpu 2.15.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
tensorflow-cpu 2.15.1 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.9.1 which is incompatible.
tensorflow-cpu 2.15.1 requires tensorflow-estimator<2.16,>=2.15.0, but you have tensorflow-estimator 2.9.0 which is incompatible.
tensorflow-text 2.15.0 requires tensorflow<2.16,>=2.15.0; platform_machine != "arm64" or platform_system != "Darwin", but you have tensorflow 2.9.3 which is incompatible.
tf2onnx 1.16.1 requires protobuf~=3.20, but

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [51]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

ImportError: cannot import name 'top_k_top_p_filtering' from 'transformers' (/home/eternal/.local/lib/python3.10/site-packages/transformers/__init__.py)

In [38]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# qa_train.json 로드
qa_train = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote 'Pride and Prejudice'?", "answer": "'Pride and Prejudice' was written by Jane Austen."}
]

# qa_test.json 로드
qa_test = [
    {"question": "What is the capital of Germany?"},
    {"question": "Who wrote 'To Kill a Mockingbird'?"}
]

# processed_wikipedia.txt 로드
documents = [
    "Paris is the capital and most populous city of France.",
    "Jane Austen is the author of 'Pride and Prejudice'.",
    "Berlin is the capital of Germany.",
    "'To Kill a Mockingbird' was written by Harper Lee."
]


In [45]:
# TF-IDF 벡터라이저 초기화 및 학습
vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform(documents)

def retrieve_document(query):
    query_vector = vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_vector, doc_vectors).flatten()
    retrieved_doc_index = cosine_similarities.argmax()
    return documents[retrieved_doc_index]



In [47]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from torch.utils.data import Dataset

# Gemma-2B-it 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16
)
class QADataset(Dataset):
    def __init__(self, qa_data, vectorizer, documents, tokenizer, max_length=512):
        self.qa_data = qa_data
        self.vectorizer = vectorizer
        self.documents = documents
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.qa_data)
    
    def __getitem__(self, idx):
        qa_pair = self.qa_data[idx]
        question = qa_pair['question']
        answer = qa_pair['answer']
        
        # Retrieve the most relevant document
        retrieved_doc = retrieve_document(question)
        
        # Prepare input text for the model
        input_text = f"Document: {retrieved_doc}\nQuestion: {question}\nAnswer:"
        inputs = self.tokenizer.encode_plus(
            input_text, 
            truncation=True, 
            padding='max_length', 
            max_length=self.max_length, 
            return_tensors='pt'
        )
        input_ids = inputs['input_ids'].squeeze()
        
        labels = self.tokenizer.encode(
            answer, 
            truncation=True, 
            padding='max_length', 
            max_length=self.max_length, 
            return_tensors='pt'
        ).squeeze()
        
        return {
            'input_ids': input_ids,
            'labels': labels
        }

# 데이터셋 생성
train_dataset = QADataset(qa_train, vectorizer, documents, tokenizer)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [48]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# 데이터 콜레이터 설정
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # 언어 모델링이 아닌 경우
)

training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=1,  # 배치 크기 줄이기
    num_train_epochs=1,  # 예시를 위해 에포크 수를 줄임
    logging_dir='./logs',
    logging_steps=10,
    gradient_accumulation_steps=8,  # Gradient Accumulation 사용
    fp16=True,  # Mixed Precision Training
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

# 모델 학습
trainer.train()


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 

In [3]:
import re
text = "I was wondering if anyone out there could enlighten me on this car."

# 길이가 1~2인 단어들을 정규 표현식을 이용하여 삭제
shortword = re.compile(r'\W*\b\w{1,2}\b')
print(shortword.sub('', text))


 was wondering anyone out there could enlighten this car.


In [5]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

import random
import numpy as np
import pandas as pd
# pytorch 사용할거라 tensorflow 임포트 안함

rand_seed = 42
random.seed(rand_seed)
np.random.seed(rand_seed)

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

print(torch.__version__)

2024-06-04 17:20:49.042369: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-04 17:20:49.117816: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2024-06-04 17:20:49.117874: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.3.0+cu121


In [9]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [10]:
from autocorrect import Speller
spell = Speller()
spell("I'm not sleapy and tehre is no place I'm giong to.")
"I'm not sleepy and there is no place I'm going to."

spell = Speller('pl')
spell('ptaaki latatją kluczmm')
'ptaki latają kluczem'

dictionary for this language not found, downloading...
__________________________________________________
couldn't download http://ipfs.io/ipfs/QmbRSZvfJV6zN12zzWhecphcvE9ZBeQdAJGQ9c9ttJXzcg/pl.tar.gz, trying next url...
__________________________________________________
couldn't download https://gateway.pinata.cloud/ipfs/QmbRSZvfJV6zN12zzWhecphcvE9ZBeQdAJGQ9c9ttJXzcg/pl.tar.gz, trying next url...
__________________________________________________
couldn't download https://cf-ipfs.com/ipfs/QmbRSZvfJV6zN12zzWhecphcvE9ZBeQdAJGQ9c9ttJXzcg/pl.tar.gz, trying next url...
__________________________________________________
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
done!


'ptaki latają kluczem'

In [13]:
spell = Speller(only_replacements=True)

In [14]:
spell = Speller(nlp_data=your_word_frequency_dict)

NameError: name 'your_word_frequency_dict' is not defined